Install required packages

In [ ]:
#r "nuget:Newtonsoft.Json, 13.0.1"

using System.Threading.Tasks;
using System.IO;
using Newtonsoft.Json;
using Newtonsoft.Json.Linq;
using System.Net;
using System.Net.Http;

Installed Packages Newtonsoft.Json, 13.0.1

Create record to hold data

In [ ]:
record City (
    string Name,
    string InseeCode,
    int Population,
    string Geometry,
    string PostCode,
    double Latitude,
    double Longitude,
    bool SruDeficit,
    decimal SocialHousingRate,
    int SocialHousingCount,
    int SnlHousingCount
);

Read the Data from GeoJSON cities file

In [ ]:
HttpClient client = new HttpClient();
var endpoint = "https://geo.api.gouv.fr/communes?codeRegion=11&fields=centre,contour,codesPostaux,population";
string response = await client.GetStringAsync(endpoint);

JArray jsonRecords = JArray.Parse(response);

Console.WriteLine("Number of cities json records : ");
Console.WriteLine(jsonRecords.Count);


Number of cities json records : 
1268


Read SRU Data

In [ ]:
JArray sruJsonRecords;
using (StreamReader reader = File.OpenText(@"bilan-sru-2020.json"))
{
    sruJsonRecords = JToken.ReadFrom(new JsonTextReader(reader)) as JArray;
}

Console.WriteLine("Number of sru cities json records : ");
Console.WriteLine(sruJsonRecords.Count);


Number of sru cities json records : 
599


Read SNL housing data

In [ ]:
JArray snlJsonRecords;
using (StreamReader reader = File.OpenText(@"snl-housing-count.json"))
{
    snlJsonRecords = JToken.ReadFrom(new JsonTextReader(reader)) as JArray;
}

Console.WriteLine("Number of SNL cities json records : ");
Console.WriteLine(snlJsonRecords.Count);


Number of SNL cities json records : 
117


Mapping to city data

In [ ]:
var result = jsonRecords.Select(jsonRecord => {
    JObject record = jsonRecord as JObject;

    string cityName = record["nom"].ToString();
    string inseeCode = record["code"].ToString();
    int population = Int32.Parse(record["population"].ToString());
    string geometry = record["contour"].ToString(Formatting.None);
    string postCode = string.Join( ",",(record["codesPostaux"] as JArray).Select(elt => elt.ToString()));
    double latitude = Double.Parse(record["centre"]["coordinates"][1].ToString());
    double longitude = Double.Parse(record["centre"]["coordinates"][0].ToString());

    JObject sruRecord = sruJsonRecords.FirstOrDefault(record => (record as JObject)["Code INSEE"].ToString() == inseeCode) as JObject;

    bool found = sruRecord != null;

    bool sruDeficit = found ? sruRecord["Commune déficitaire"].ToString() == "OUI" : false;

    decimal socialHousingRate = found ?
        Decimal.Parse(
            sruRecord["Taux LLS au 1er janvier 2019"]
            .ToString()
            .Replace("%","")
            .Trim()) 
        :
        -1;
    int socialHousingCount = found ? Int32.Parse(sruRecord["Total LLS"].ToString()) : -1;
    
    JObject snlRecord = snlJsonRecords.FirstOrDefault(record => (cityName == "Paris" ? "75000" : postCode) == record["postCode"].ToString()) as JObject;

    int snlHousingCount = snlRecord == null ? 0 :  Int32.Parse(snlRecord["snlHousingCount"].ToString());

    return new City(
        cityName, 
        inseeCode, 
        population,
        geometry,
        postCode, 
        latitude, 
        longitude,
        sruDeficit,
        socialHousingRate,
        socialHousingCount,
        snlHousingCount
    );
});


Export to CSV

In [ ]:
List<string> csvLines = new List<string> { "City Name;Insee Code;Population;Geometry;PostCode;Latitude;Longitude;Sru Deficit;Social Housing Rate;Social Housing Count;SNL Housing Count" };
csvLines.AddRange(result.Select((city) => $"{city.Name};{city.InseeCode};{city.Population};{city.Geometry};{city.PostCode};{city.Latitude};{city.Longitude};{city.SruDeficit};{city.SocialHousingRate};{city.SocialHousingCount};{city.SnlHousingCount}"));
await File.WriteAllLinesAsync("cities.csv", csvLines);